# SemCHILDES construction

As an initial effort to construct SemCHILDES, I use an automatic word sense disambiguation to annotate the CHILDES corpus with word senses. As future work, I intend to manually annotate part of the corpus and then evaluate the performance of different algorithms, approaches, and combination of them in the annotated data. The best approach will be used to annotate the entire corpus.

Used tool: PySupWSDPocket - https://github.com/rodriguesfas/PySupWSDPocket




## PySupWSDPocket

PySupWSDPocket is a python lib for the [SupWSD Pocket](https://supwsd.net/supwsd/pocket.jsp). SupWSD is a supervised model for Word Sense Disambiguation.

We install it from github to get the latest version.

https://drive.google.com/file/d/1hEMlbToLL4xN7HJhPtebMbKYeethWmha/view?usp=sharing

In [7]:
!pip install git+https://github.com/rodriguesfas/PySupWSDPocket.git

  Cloning https://github.com/rodriguesfas/PySupWSDPocket.git to /tmp/pip-req-build-yi9rlw9t
  Running command git clone -q https://github.com/rodriguesfas/PySupWSDPocket.git /tmp/pip-req-build-yi9rlw9t
  Stored in directory: /tmp/pip-ephem-wheel-cache-q1ltryik/wheels/60/71/8d/80f8c9ddf9fd2b65d10328afb6d580cfd83e4fbbc690cfb4dc
Successfully built pysupwsdpocket


PySupWSDPocket requires downloading its ~2GB model available on https://supwsd.net/supwsd/downloads.jsp#supwsd_pocket.

In [3]:
!pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102kB 29.2MB/s ta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/42/7e/df/c83c65b50523efb0cc81af5e592f7cb8138931f80f5ba1f6b9
Successfully built gdown


In [4]:
!mkdir pysupwsdpocket_models
!gdown  https://drive.google.com/uc?id=1hEMlbToLL4xN7HJhPtebMbKYeethWmha  -O="pysupwsdpocket_models/en.zip"

mkdir: cannot create directory ‘pysupwsdpocket_models’: File exists
Downloading...
From: https://drive.google.com/uc?id=1hEMlbToLL4xN7HJhPtebMbKYeethWmha
To: /root/capsule/pysupwsdpocket_models/en.zip
100%|███████████████████████████████████████| 1.80G/1.80G [00:13<00:00, 131MB/s]


In [9]:
from pysupwsdpocket import PySupWSDPocket
nlp = PySupWSDPocket(lang='en', model='semcor_omsti', model_path="./pysupwsdpocket_models/")    

## CHILDES

The Child Language Data Exchange System (CHILDES) is a corpus established in 1984 by Brian MacWhinney and Catherine Snow to serve as a central repository for data of first language acquisition[¹](https://en.wikipedia.org/wiki/CHILDES). It counts with a list of different corpora from many languages that can be downloaded in XML or CHA format.

In this notebook we download only one corpus, but SemCHILDES is composed by the entire American English CHILDES.

In [10]:
!mkdir corpora
corpora_files = ["Bates.zip", "Bernstein.zip", "Bliss.zip", "Bloom.zip", "Bohannon.zip", "Braunwald.zip", "Brent.zip", "Brown.zip", "Clark.zip", "Demetras1.zip", "Demetras2.zip", "Evans.zip", "Feldman.zip", "Garvey.zip", "Gathercole.zip", "Gelman.zip", "Gleason.zip", "Gopnik.zip", "HSLLD.zip", "Haggerty.zip", "Hall.zip", "Hicks.zip", "Higginson.zip", "Kuczaj.zip", "MacWhinney.zip", "McCune.zip", "McMillan.zip", "Morisset.zip", "Nelson.zip", "NewEngland.zip", "NewmanRatner.zip", "Peters.zip", "PetersonMcCabe.zip", "Post.zip", "Rollins.zip", "Sachs.zip", "Sawyer.zip", "Snow.zip", "Soderstrom.zip", "Sprott.zip", "Suppes.zip", "Tardif.zip", "Valian.zip", "VanHouten.zip", "VanKleeck.zip", "Warren.zip", "Weist.zip"]
for corpus_file in corpora_files:
    !wget https://childes.talkbank.org/data-xml/Eng-NA/$corpus_file -O corpora/$corpus_file

In [8]:
for corpus_file in corpora_files:
    !unzip corpora/$corpus_file -d corpora

Archive:  corpora/Bates.zip
  inflating: corpora/Bates/Free28/mandy28.xml  
  inflating: corpora/Bates/Free28/doug28.xml  
  inflating: corpora/Bates/Free28/frank28.xml  
  inflating: corpora/Bates/Free28/ivy28.xml  
  inflating: corpora/Bates/Free28/paula28.xml  
  inflating: corpora/Bates/Free28/ed28.xml  
  inflating: corpora/Bates/Free28/hank28.xml  
  inflating: corpora/Bates/Free28/pete28.xml  
  inflating: corpora/Bates/Free28/will28.xml  
  inflating: corpora/Bates/Free28/sue28.xml  
  inflating: corpora/Bates/Free28/rick28.xml  
  inflating: corpora/Bates/Free28/amy28.xml  
  inflating: corpora/Bates/Free28/betty28.xml  
  inflating: corpora/Bates/Free28/chuck28.xml  
  inflating: corpora/Bates/Free28/steve28.xml  
  inflating: corpora/Bates/Free28/olivia28.xml  
  inflating: corpora/Bates/Free28/keith28.xml  
  inflating: corpora/Bates/Free28/ruth28.xml  
  inflating: corpora/Bates/Free28/wanda28.xml  
  inflating: corpora/Bates/Free28/george28.xml  
  inflating: corpora/Bate

  inflating: corpora/Bloom/Peter/020415.xml  
  inflating: corpora/Bloom/Peter/020303.xml  
  inflating: corpora/Bloom/Peter/020915.xml  
  inflating: corpora/Bloom/Peter/010908.xml  
  inflating: corpora/Bloom/Peter/011011.xml  
  inflating: corpora/Bloom/Peter/020213.xml  
  inflating: corpora/Bloom/Peter/020713.xml  
  inflating: corpora/Bloom/Peter/020503.xml  
  inflating: corpora/Bloom/Peter/011117.xml  
  inflating: corpora/Bloom/Peter/011105.xml  
  inflating: corpora/Bloom/Peter/020118.xml  
  inflating: corpora/Bloom/Peter/021019.xml  
  inflating: corpora/Bloom/Peter/020324.xml  
  inflating: corpora/Bloom/Peter/020616.xml  
  inflating: corpora/Bloom/Peter/030120.xml  
  inflating: corpora/Bloom/Peter/020812.xml  
  inflating: corpora/Bloom/Peter/020522.xml  
Archive:  corpora/Bohannon.zip
  inflating: corpora/Bohannon/Nat/angela.xml  
  inflating: corpora/Bohannon/Nat/doug.xml  
  inflating: corpora/Bohannon/Nat/tom.xml  
  inflating: corpora/Bohannon/Nat/jim.xml  
  infla

  inflating: corpora/Braunwald/020616.xml  
  inflating: corpora/Braunwald/030316a.xml  
  inflating: corpora/Braunwald/020603.xml  
  inflating: corpora/Braunwald/010614b.xml  
  inflating: corpora/Braunwald/010720.xml  
  inflating: corpora/Braunwald/010600b.xml  
  inflating: corpora/Braunwald/010607.xml  
  inflating: corpora/Braunwald/010902.xml  
  inflating: corpora/Braunwald/010614a.xml  
  inflating: corpora/Braunwald/030616.xml  
  inflating: corpora/Braunwald/030000.xml  
  inflating: corpora/Braunwald/010604b.xml  
  inflating: corpora/Braunwald/020424.xml  
  inflating: corpora/Braunwald/030009.xml  
  inflating: corpora/Braunwald/030627a.xml  
  inflating: corpora/Braunwald/020300b.xml  
  inflating: corpora/Braunwald/010511.xml  
  inflating: corpora/Braunwald/041000.xml  
  inflating: corpora/Braunwald/040214.xml  
  inflating: corpora/Braunwald/020710.xml  
  inflating: corpora/Braunwald/030622.xml  
  inflating: corpora/Braunwald/030618b.xml  
  inflating: corpora/Bra

  inflating: corpora/Braunwald/0diary/010707.xml  
  inflating: corpora/Braunwald/0diary/020818.xml  
  inflating: corpora/Braunwald/0diary/010324.xml  
  inflating: corpora/Braunwald/0diary/020830.xml  
  inflating: corpora/Braunwald/0diary/020911.xml  
  inflating: corpora/Braunwald/0diary/010529.xml  
  inflating: corpora/Braunwald/0diary/020115.xml  
  inflating: corpora/Braunwald/0diary/010717.xml  
  inflating: corpora/Braunwald/0diary/020117.xml  
  inflating: corpora/Braunwald/0diary/010406.xml  
  inflating: corpora/Braunwald/0diary/010416.xml  
  inflating: corpora/Braunwald/0diary/030122.xml  
  inflating: corpora/Braunwald/0diary/020209.xml  
  inflating: corpora/Braunwald/0diary/020023.xml  
  inflating: corpora/Braunwald/0diary/020722.xml  
  inflating: corpora/Braunwald/0diary/020914.xml  
  inflating: corpora/Braunwald/0diary/020124.xml  
  inflating: corpora/Braunwald/0diary/010916.xml  
  inflating: corpora/Braunwald/0diary/010425.xml  
  inflating: corpora/Braunwald/

  inflating: corpora/Braunwald/0diary/020021.xml  
  inflating: corpora/Braunwald/0diary/010805.xml  
  inflating: corpora/Braunwald/0diary/011126.xml  
  inflating: corpora/Braunwald/0diary/010725.xml  
  inflating: corpora/Braunwald/0diary/021126.xml  
  inflating: corpora/Braunwald/0diary/011006.xml  
  inflating: corpora/Braunwald/0diary/010304.xml  
  inflating: corpora/Braunwald/0diary/011016.xml  
  inflating: corpora/Braunwald/0diary/020522.xml  
  inflating: corpora/Braunwald/0diary/030429.xml  
  inflating: corpora/Braunwald/0diary/020716.xml  
  inflating: corpora/Braunwald/0diary/020728.xml  
  inflating: corpora/Braunwald/0diary/020110.xml  
Archive:  corpora/Brent.zip
  inflating: corpora/Brent/q1/010217b.xml  
  inflating: corpora/Brent/q1/010205.xml  
  inflating: corpora/Brent/q1/010120.xml  
  inflating: corpora/Brent/q1/000928.xml  
  inflating: corpora/Brent/q1/010009.xml  
  inflating: corpora/Brent/q1/010113.xml  
  inflating: corpora/Brent/q1/001128.xml  
  infla

...

  inflating: corpora/Rollins/ds09.xml  
  inflating: corpora/Rollins/nb12.xml  
  inflating: corpora/Rollins/nb09.xml  
  inflating: corpora/Rollins/cy12.xml  
  inflating: corpora/Rollins/zx09.xml  
  inflating: corpora/Rollins/ds06.xml  
  inflating: corpora/Rollins/ax06.xml  
  inflating: corpora/Rollins/st12.xml  
  inflating: corpora/Rollins/nj09.xml  
  inflating: corpora/Rollins/mm09.xml  
  inflating: corpora/Rollins/te12.xml  
  inflating: corpora/Rollins/ps09.xml  
  inflating: corpora/Rollins/gb06.xml  
  inflating: corpora/Rollins/et06.xml  
  inflating: corpora/Rollins/ma12.xml  
  inflating: corpora/Rollins/di06.xml  
  inflating: corpora/Rollins/sa06.xml  
  inflating: corpora/Rollins/gp06.xml  
  inflating: corpora/Rollins/sb06.xml  
  inflating: corpora/Rollins/pa06.xml  
  inflating: corpora/Rollins/tx09.xml  
  inflating: corpora/Rollins/cb06.xml  
  inflating: corpora/Rollins/tl06.xml  
  inflating: corpora/Rollins/jp18.xml  
  inflating: corpora/Rollins/cy30.xml  


  inflating: corpora/Sawyer/2-28-92.xml  
Archive:  corpora/Snow.zip
  inflating: corpora/Snow/020804b.xml  
  inflating: corpora/Snow/030410a.xml  
  inflating: corpora/Snow/030106a.xml  
  inflating: corpora/Snow/030106b.xml  
  inflating: corpora/Snow/030904.xml  
  inflating: corpora/Snow/020622a.xml  
  inflating: corpora/Snow/030418c.xml  
  inflating: corpora/Snow/020804a.xml  
  inflating: corpora/Snow/020600c.xml  
  inflating: corpora/Snow/020622b.xml  
  inflating: corpora/Snow/030408a.xml  
  inflating: corpora/Snow/030409b.xml  
  inflating: corpora/Snow/030019a.xml  
  inflating: corpora/Snow/030408c.xml  
  inflating: corpora/Snow/030019b.xml  
  inflating: corpora/Snow/020518b.xml  
  inflating: corpora/Snow/030408d.xml  
  inflating: corpora/Snow/030408b.xml  
  inflating: corpora/Snow/021100c.xml  
  inflating: corpora/Snow/020819b.xml  
  inflating: corpora/Snow/020600a.xml  
  inflating: corpora/Snow/030021b.xml  
  inflating: corpora/Snow/030409d.xml  
  inflating:

  inflating: corpora/Suppes/020913.xml  
Archive:  corpora/Tardif.zip
  inflating: corpora/Tardif/e21.xml  
  inflating: corpora/Tardif/e08.xml  
  inflating: corpora/Tardif/e19.xml  
  inflating: corpora/Tardif/e16.xml  
  inflating: corpora/Tardif/e11.xml  
  inflating: corpora/Tardif/e06.xml  
  inflating: corpora/Tardif/e10.xml  
  inflating: corpora/Tardif/e26book.xml  
  inflating: corpora/Tardif/e03.xml  
  inflating: corpora/Tardif/e12.xml  
  inflating: corpora/Tardif/e24.xml  
  inflating: corpora/Tardif/e25.xml  
  inflating: corpora/Tardif/e13.xml  
  inflating: corpora/Tardif/e17.xml  
  inflating: corpora/Tardif/e04.xml  
  inflating: corpora/Tardif/e15.xml  
  inflating: corpora/Tardif/e18.xml  
  inflating: corpora/Tardif/e20.xml  
  inflating: corpora/Tardif/e23.xml  
  inflating: corpora/Tardif/e09.xml  
  inflating: corpora/Tardif/e02.xml  
  inflating: corpora/Tardif/e22.xml  
  inflating: corpora/Tardif/e01.xml  
  inflating: corpora/Tardif/e14.xml  
  inflating: c

Archive:  corpora/VanKleeck.zip
  inflating: corpora/VanKleeck/megan1.xml  
  inflating: corpora/VanKleeck/rachel2.xml  
  inflating: corpora/VanKleeck/lara2.xml  
  inflating: corpora/VanKleeck/walter2a.xml  
  inflating: corpora/VanKleeck/matjoy2.xml  
  inflating: corpora/VanKleeck/nikki1b.xml  
  inflating: corpora/VanKleeck/graham2.xml  
  inflating: corpora/VanKleeck/ben2.xml  
  inflating: corpora/VanKleeck/nikki2b.xml  
  inflating: corpora/VanKleeck/jessica1.xml  
  inflating: corpora/VanKleeck/justin1a.xml  
  inflating: corpora/VanKleeck/justin2.xml  
  inflating: corpora/VanKleeck/lara1.xml  
  inflating: corpora/VanKleeck/graham1.xml  
  inflating: corpora/VanKleeck/megan2.xml  
  inflating: corpora/VanKleeck/bree2.xml  
  inflating: corpora/VanKleeck/susan1.xml  
  inflating: corpora/VanKleeck/nikki1a.xml  
  inflating: corpora/VanKleeck/ben1.xml  
  inflating: corpora/VanKleeck/mattm2.xml  
  inflating: corpora/VanKleeck/matjoy1.xml  
  inflating: corpora/VanKleeck/jessi

  inflating: corpora/Weist/Matt/030002.xml  
  inflating: corpora/Weist/Matt/020407.xml  
  inflating: corpora/Weist/Matt/030226.xml  
  inflating: corpora/Weist/Matt/040809.xml  
  inflating: corpora/Weist/Matt/020317.xml  
  inflating: corpora/Weist/Matt/031106.xml  
  inflating: corpora/Weist/Matt/040610.xml  
  inflating: corpora/Weist/Matt/030514.xml  
  inflating: corpora/Weist/Matt/040704.xml  
  inflating: corpora/Weist/Matt/040405.xml  
  inflating: corpora/Weist/Matt/030704.xml  
  inflating: corpora/Weist/Jillian/020916.xml  
  inflating: corpora/Weist/Jillian/020122.xml  
  inflating: corpora/Weist/Jillian/020421.xml  
  inflating: corpora/Weist/Jillian/020507.xml  
  inflating: corpora/Weist/Jillian/020707.xml  
  inflating: corpora/Weist/Jillian/020325.xml  
  inflating: corpora/Weist/Jillian/020129.xml  
  inflating: corpora/Weist/Jillian/020101.xml  
  inflating: corpora/Weist/Jillian/020521.xml  
  inflating: corpora/Weist/Jillian/020526.xml  
  inflating: corpora/Weis

### Extract data from CHILDES

The data extraction is made by parsing the CHILDES' XML files.

In [1]:
import os
from glob import glob
PATH = "./corpora"
all_files = [y for x in os.walk(PATH) for y in glob(os.path.join(x[0], '*.xml'))]
print(len(all_files))
print(all_files[0])

7719
./corpora/Suppes/030107.xml


#### Find participants

This information is important for making queries in the future. For example, get sentences by children age.

In [2]:
def find_participants(root):
    participants = []
    for participant in root.find(ns+"Participants"):
      participants.append(participant.attrib)
    return participants

#### Parse utterances

In [3]:
def parse_utterance(u):
    wsd_doc = []
    if 'text' in u: # some utterances in CHILDES have just researchers comments or actions like (he screamed)
        doc = nlp.wsd(u['text'])
        for token in doc.tokens():
          wsd_doc.append(token.__dict__)
    return wsd_doc

#### Process utterances

In [4]:
from tqdm import tqdm
def process_utterances(root, process_faster=False):
    utterances = []
    for u in root.findall(ns+'u'):
      utterance_dict = u.attrib
      utterance_dict['original_tokens'] = []
      tokens = []
      for token in u.getchildren():
        if token.tag == ns+"w":
          tags = [a.tag for a in token.getchildren()]
          if ns+"shortening" in tags:
            try:
                tokens.append(token.find(ns+'mor').find(ns+"mw").find(ns+"stem").text)
            except:
                pass
          elif token.text is not None:
            tokens.append(token.text)
        elif token.tag == ns+"g": # group of words
          token = token.find(ns+'w')
          if token is not None:
              tags = [a.tag for a in token.getchildren()]
              if ns+"shortening" in tags:
                try:
                    tokens.append(token.find(ns+'mor').find(ns+"mw").find(ns+"stem").text)
                except:
                    pass
              elif token.text is not None:
                tokens.append(token.text)

        elif token.tag == ns+"t": # punctuation
          if token.attrib['type'] == 'p':
            tokens.append(".")
          elif token.attrib['type'] == 'q':
            tokens.append("?")
        elif token.tag == ns+"tagMarker": #comma
          tokens.append(',')
      if len(tokens) > 1:
        utterance_dict['text'] = " ".join(tokens)
      if not process_faster:
          utterance_dict['wsd_doc'] = parse_utterance(utterance_dict)
      utterances.append(utterance_dict)

    return utterances

In [22]:
!mkdir dicts

In [5]:
import xml.etree.ElementTree as ET
from tqdm import tqdm
import warnings
import json
from os.path import exists

warnings.filterwarnings('ignore')
all_dicts = []

process_n_files = len(all_files) # change to len(all_files) to use all

faster_processing = True # To process faster, you can use pysupwsd process_corpus method. However, by using this method we cannot use the sentences metadata (e.g., children age).
if faster_processing:
    !mkdir only_texts

for xml_file in tqdm(all_files):
    if faster_processing:
        txt_file = "only_texts/{0}.txt".format("_".join(xml_file.split('/')[-2:]))
        if exists(txt_file):
            continue
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = "{http://www.talkbank.org/ns/talkbank}"
    sem_dict = root.attrib
    sem_dict['file'] = "/content/corpora/MacWhinney/030018a.xml"
    sem_dict['participants'] = find_participants(root)
    sem_dict['utterances'] = process_utterances(root,faster_processing)
    
    if faster_processing:
        ft = open("only_texts/{0}.txt".format("_".join(xml_file.split('/')[-2:])),'w')
        ft.writelines([l['text']+"\n" for l in sem_dict['utterances'] if 'text' in l])
        ft.close() 
    
    all_dicts.append(sem_dict)
    json.dump(sem_dict, open("dicts/{0}.json".format("_".join(xml_file.split('/')[-2:])),'w'))

mkdir: cannot create directory ‘only_texts’: File exists


100%|██████████| 7719/7719 [00:49<00:00, 154.78it/s]


#### Create corpus for BERT input

In [11]:
!pip install nltk

     |████████████████████████████████| 1.5MB 28.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 48.0MB/s ta 0:00:01
     |████████████████████████████████| 675kB 60.3MB/s eta 0:00:01
     |████████████████████████████████| 307kB 69.9MB/s eta 0:00:01


In [12]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [23]:
import os
from glob import glob
f = open("data/semCHILDES.txt",'w')
if faster_processing:
    PATH = "only_texts/"
    only_texts_files = [y for x in os.walk(PATH) for y in glob(os.path.join(x[0], '*.txt'))]
    
    for text_file in tqdm(only_texts_files):
        corpus = nlp.parse_corpus(text_file)
        for doc in corpus:
            new_sentence = []
            for t in doc.tokens():
              token = t.__dict__
              if token['word'] in ['me','and','or',',']:
                  new_sentence.append(token['word'])
              elif token['lemma'] in ["can","a","to","how","what",'this',"that"]:
                  new_sentence.append(token['lemma'])
              elif token['senses'][0]['id'] != 'U':
                  new_sentence.append(token['senses'][0]['id'])
              elif token['pos'] in ['IN','PRP','.','WRB','CC',"PRP$","DT"]:
                  new_sentence.append(token['lemma'])
              elif token['pos'] in ['NNP']:
                  new_sentence.append('proper_noun')
              elif token['pos'] in ['NN',"NNS"]:
                  n_token = None
                  synsets = wn.synsets(token['lemma'],'n')
                  if len(synsets) > 0:
                      synset = synsets[0]
                      for l in synset.lemmas():
                          if l.name() == token['lemma']:
                              n_token = l.key()
                  if n_token is not None:
                    new_sentence.append(n_token)
                  else:
                    new_sentence.append(token['lemma']) # it may be words that are common on children vocabulary.
            if len(new_sentence) > 1:
                f.write(" ".join(new_sentence)+"\n")
f.close()




  0%|          | 0/7142 [00:00<?, ?it/s]

  0%|          | 1/7142 [00:12<24:09:30, 12.18s/it]

  0%|          | 2/7142 [00:14<18:29:38,  9.32s/it]

  0%|          | 3/7142 [00:25<19:29:45,  9.83s/it]

  0%|          | 4/7142 [00:46<26:00:23, 13.12s/it]

  0%|          | 5/7142 [01:16<35:55:39, 18.12s/it]

  0%|          | 6/7142 [01:18<26:15:51, 13.25s/it]

  0%|          | 7/7142 [01:23<21:23:37, 10.79s/it]

  0%|          | 8/7142 [01:29<18:24:53,  9.29s/it]

  0%|          | 9/7142 [01:32<14:36:02,  7.37s/it]

  0%|          | 10/7142 [01:34<11:39:06,  5.88s/it]

  0%|          | 11/7142 [01:40<11:40:37,  5.90s/it]

  0%|          | 12/7142 [01:46<11:41:00,  5.90s/it]

  0%|          | 13/7142 [01:57<15:04:35,  7.61s/it]

  0%|          | 14/7142 [01:58<11:09:49,  5.64s/it]

  0%|          | 15/7142 [02:01<9:03:01,  4.57s/it] 

  0%|          | 16/7142 [02:09<11:04:28,  5.59s/it]

  0%|          | 17/7142 [02:29<19:53:50, 10.05s/it]

  0%|          | 18/7142 [04:44<93:59:32, 47.50

  2%|▏         | 136/7142 [22:34<26:49:28, 13.78s/it]

  2%|▏         | 137/7142 [22:35<19:22:25,  9.96s/it]

  2%|▏         | 138/7142 [22:39<16:03:00,  8.25s/it]

  2%|▏         | 139/7142 [23:08<28:23:27, 14.59s/it]

  2%|▏         | 140/7142 [23:17<24:51:13, 12.78s/it]

  2%|▏         | 141/7142 [23:31<25:36:58, 13.17s/it]

  2%|▏         | 142/7142 [23:33<19:02:46,  9.80s/it]

  2%|▏         | 143/7142 [23:37<15:38:11,  8.04s/it]

  2%|▏         | 144/7142 [23:39<12:11:56,  6.28s/it]

  2%|▏         | 145/7142 [23:40<9:08:48,  4.71s/it] 

  2%|▏         | 146/7142 [23:47<10:30:34,  5.41s/it]

  2%|▏         | 147/7142 [23:48<7:57:19,  4.09s/it] 

  2%|▏         | 148/7142 [24:00<12:36:18,  6.49s/it]

  2%|▏         | 149/7142 [24:05<11:30:40,  5.93s/it]

  2%|▏         | 150/7142 [24:12<12:12:55,  6.29s/it]

  2%|▏         | 151/7142 [24:16<10:47:06,  5.55s/it]

  2%|▏         | 152/7142 [24:17<8:09:16,  4.20s/it] 

  2%|▏         | 153/7142 [24:20<7:26:15,  3.83s/it]

  2%|▏     

  4%|▍         | 270/7142 [41:39<14:28:28,  7.58s/it]

  4%|▍         | 271/7142 [42:15<30:45:23, 16.11s/it]

  4%|▍         | 272/7142 [42:28<29:12:39, 15.31s/it]

  4%|▍         | 273/7142 [42:30<21:22:14, 11.20s/it]

  4%|▍         | 274/7142 [42:31<15:32:49,  8.15s/it]

  4%|▍         | 275/7142 [42:32<11:31:05,  6.04s/it]

  4%|▍         | 276/7142 [42:43<14:33:23,  7.63s/it]

  4%|▍         | 277/7142 [42:51<14:29:00,  7.60s/it]

  4%|▍         | 278/7142 [42:57<13:21:32,  7.01s/it]

  4%|▍         | 279/7142 [43:05<14:13:49,  7.46s/it]

  4%|▍         | 280/7142 [43:06<10:32:37,  5.53s/it]

  4%|▍         | 281/7142 [43:09<8:57:51,  4.70s/it] 

  4%|▍         | 282/7142 [43:11<7:15:08,  3.81s/it]

  4%|▍         | 283/7142 [43:18<9:11:29,  4.82s/it]

  4%|▍         | 284/7142 [43:19<7:05:37,  3.72s/it]

  4%|▍         | 285/7142 [43:23<7:12:33,  3.79s/it]

  4%|▍         | 286/7142 [43:34<11:15:07,  5.91s/it]

  4%|▍         | 287/7142 [43:38<10:01:28,  5.26s/it]

  4%|▍        

  6%|▌         | 404/7142 [1:00:47<13:52:40,  7.41s/it]

  6%|▌         | 405/7142 [1:00:51<12:25:20,  6.64s/it]

  6%|▌         | 406/7142 [1:01:09<18:45:00, 10.02s/it]

  6%|▌         | 407/7142 [1:01:33<26:30:10, 14.17s/it]

  6%|▌         | 408/7142 [1:01:38<21:13:31, 11.35s/it]

  6%|▌         | 409/7142 [1:01:48<20:21:55, 10.89s/it]

  6%|▌         | 410/7142 [1:02:03<22:37:03, 12.09s/it]

  6%|▌         | 411/7142 [1:02:22<26:37:15, 14.24s/it]

  6%|▌         | 412/7142 [1:02:28<21:47:56, 11.66s/it]

  6%|▌         | 413/7142 [1:02:29<15:51:08,  8.48s/it]

  6%|▌         | 414/7142 [1:02:33<13:43:39,  7.35s/it]

  6%|▌         | 415/7142 [1:02:45<16:00:23,  8.57s/it]

  6%|▌         | 416/7142 [1:02:47<12:18:32,  6.59s/it]

  6%|▌         | 417/7142 [1:02:48<9:22:35,  5.02s/it] 

  6%|▌         | 418/7142 [1:03:11<19:34:20, 10.48s/it]

  6%|▌         | 419/7142 [1:03:14<14:56:25,  8.00s/it]

  6%|▌         | 420/7142 [1:03:17<12:34:23,  6.73s/it]

  6%|▌         | 421/7142 [1:03

KeyboardInterrupt: 

In [ ]:
f = open("data/semCHILDES.txt",'w')
if not faster_processing:
    for sem_dict in all_dicts:
        for u in tqdm(sem_dict['utterances']):
          if 'wsd_doc' in u:
            new_sentence = []
            for token in u['wsd_doc']:
              if token['word'] in ['me','and','or',',']:
                  new_sentence.append(token['word'])
              elif token['lemma'] in ["can","a","to","how","what",'this',"that"]:
                  new_sentence.append(token['lemma'])
              elif token['senses'][0]['id'] != 'U':
                  new_sentence.append(token['senses'][0]['id'])
              elif token['pos'] in ['IN','PRP','.','WRB','CC',"PRP$","DT"]:
                  new_sentence.append(token['lemma'])
              elif token['pos'] in ['NNP']:
                  new_sentence.append('proper_noun')
              elif token['pos'] in ['NN',"NNS"]:
                  n_token = None
                  synsets = wn.synsets(token['lemma'],'n')
                  if len(synsets) > 0:
                      synset = synsets[0]
                      for l in synset.lemmas():
                          if l.name() == token['lemma']:
                              n_token = l.key()
                  if n_token is not None:
                    new_sentence.append(n_token)
                  else:
                    new_sentence.append(token['lemma']) # it may be words that are common on children vocabulary.
            if len(new_sentence) > 1:
                f.write(" ".join(new_sentence)+"\n")
    f.close()
